In [4]:
import pandas as pd

In [5]:
user = pd.read_csv('user.csv')

In [57]:
user_session = pd.read_csv('user_session.csv')
user['last_visited'] = pd.to_datetime(user['last_visited'])
user_session['date'] = pd.to_datetime(user_session['date'])
user_session['date_only'] = user_session['date'].dt.date 

## Задача 1

Посчитайте долю пользователей мужского пола в таблице `user`.

Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для подсчёта доли объектов, удовлетворяющих заданному условию, удобно использовать функцию `mean`, вызвав её от колонки-фильтра со значениями булевого типа. В данном случае каждое знаечние `True` будет приведено к $1$, а каждое значение `False` — к $0$. Усреднение этих значений как раз и даст долю объектов, удволетворяющих заданному условию.

In [9]:
display(user)
display(user_session)
print((user['gender'] == 'М').mean())

,id,gender,birthday,region,last_visited
0,1,М,2007-07-15 18:53:02,Город Москва,2023-09-20 18:00:34.464576
1,2,Ж,2007-06-16 11:48:48,Краснодарский край,2023-10-04 16:01:11.464583
2,3,М,2010-08-09 14:11:20,Город Санкт-Петербург,2023-09-27 17:14:43.464586
3,4,М,2008-07-03 10:50:41,Город Санкт-Петербург,2023-09-27 13:46:11.464589
4,5,М,2010-02-15 23:03:02,Краснодарский край,2023-10-12 10:16:43.464591
...,...,...,...,...,...
995,996,М,2009-10-23 06:33:42,Город Москва,2023-10-11 15:27:50.466992
996,997,М,2008-11-27 05:44:59,Город Москва,2023-10-16 12:01:08.466994
997,998,М,2008-12-16 14:50:57,Краснодарский край,2023-10-13 17:29:46.466996
998,999,Ж,2009-09-28 13:25:52,Краснодарский край,2023-09-23 15:38:04.466999


,session_id,user_id,date,time_watched
0,1,558,2023-01-23 00:52:14,10
1,2,435,2023-01-06 20:59:45,51
2,3,783,2023-01-27 19:24:25,2
3,4,575,2023-01-23 21:55:54,29
4,5,598,2023-01-22 15:06:09,5
...,...,...,...,...
52607,52608,490,2023-10-07 09:03:03,14
52608,52609,527,2023-10-10 17:48:33,35
52609,52610,2,2023-10-01 04:12:31,37
52610,52611,939,2023-10-01 00:47:51,23


0.425


## Задача 2

Посчитайте долю пользователей из Краснодарского края, которые заходили на платформу в сентябре $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [22]:
# Объединяем данные
course_region = user_session.merge(user, left_on='user_id', right_on='id')

# Фильтруем только пользователей из Краснодарского края
krasnodar = course_region[course_region['region'] == 'Краснодарский край']

# Фильтруем сессии — сентябрь 2023
september_sessions = krasnodar[
    (krasnodar['last_visited'].dt.year == 2023) &
    (krasnodar['last_visited'].dt.month == 9)
]

# Находим уникальных пользователей из Краснодарского края, заходивших в сентябре
users_in_september = september_sessions['user_id'].nunique()

# Общее число пользователей из Краснодарского края
total_krasnodar = krasnodar['user_id'].nunique()

# Считаем долю
fraction = users_in_september / total_krasnodar

# Округляем до 3 знаков после запятой
print(round(fraction, 3))

0.401


## Задача 3

По таблице `user_session` рассчитайте долю активных пользователей в мае $2023$-го года относительно общего числа пользователей платформы из таблицы `user`.

Активными считайте пользователей, у которых в этот месяц была хотя бы одна сессия.

Ответ округлите до $2$-х знаков после запятой.

In [28]:
may_session = user_session[(user_session['date'].dt.year == 2023) &
    (user_session['date'].dt.month == 5)]
unique = may_session['user_id'].nunique()
print(len(unique)/len())

## Задача 4

Дадим другое определение активным пользователям. Будем считать активными в конкретном месяце тех, кто посмотрел хотя бы $60$ минут видеоконтента в этом месяце.

Рассчитайте долю активных пользователей среди тех, у кого была хотя бы одна сессия в марте $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [36]:
may_session = user_session[(user_session['date'].dt.year == 2023) &
    (user_session['date'].dt.month == 3)]
users_march = may_session['user_id'].nunique()
mrch = may_session.groupby(['user_id'], as_index=False).agg({'time_watched': 'sum'})
print((mrch['time_watched'] >=60).mean())

0.9192872117400419


# Задачи 5-9

Рассчитайте, сколько в среднем смотрят видеоконтента пользователи за один день активности. Днём активности называем день, в который у пользователя была сессия.

## Задача 5

Для этого сначала вычислите суммарное число дней, когда пользователи смотрели видеоконтент.

Обратите внимание, что хараткеристику можно посчитать двумя способами. Рассчитатйте её как число уникальных пар вида (пользователь; день, когда у него была сессия).

In [47]:
total_days = (user_session
              .groupby('user_id')['date_only']
              .nunique()
              .sum())
print(total_days
     )

44932


## Задача 6

Теперь вычислите суммарное время просмотра видеоконтента пользователями.

In [48]:
print(user_session['time_watched'].sum())

1607552


## Задача 7

Вычислите среднее количество просмотренного пользователями видеоконтента за день активности, как отношение суммарного времени просмотра к суммарному числу дней просмотра.

Ответ округлите до $2$-х знаков после запятой.

In [49]:
print(user_session['time_watched'].sum()/total_days)

35.77744146710585


## Задача 8

Обратите внимание, что есть альтернативный способ рассчитать, сколько в среднем видеоконтента смотрят пользователи за один день активности.

Во-первых, рассчитайте для каждого из пользователей среднее количество просмотренного им видеоконтента. Принцип вычисления метрики для конкретного пользователя такой же, как был для всех пользователей вместе: нужно взять суммарное время просмотра контента и разделить на суммарное количество уникальных дней активности.

В качестве ответа укажите среднее количество просмотренного видеоконтента пользователем с `id`, равным $738$. Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для того чтобы вычислить среднее количество видеоконтента, просмотренного конкретным пользователем, удобно провести группировку по пользователям и произвести агрегацию по двум колонкам отдельно: по колонке `time_watched`, просуммировав значения в ней , и по колонке `date`, посчитав количество уникальных дней активности для каждого из пользователей.

Для того чтобы произвести агрегацию по колонке `date`, удобно воспользоваться лямбда-функцией, которая сначала произведёт округление моментов времени из колонке `date` _вниз_ до дней, а потом посчитает количество полученных уникальных дней.

In [64]:
user_stats = user_session.groupby('user_id').agg(
    total_time=('time_watched', 'sum'),
    active_days=('date', lambda x: x.dt.date.nunique())  # число уникальных дней
)

# Среднее время просмотра в день активности
user_stats['avg_per_active_day'] = user_stats['total_time'] / user_stats['active_days']
print(user_stats['avg_per_active_day'].mean())

34.793012875603004


## Задача 9

Для того чтобы вычислить среднее время просмотра видеоконтента за день активности, посчитайте среднее арифметическое данной характеристики по всем пользователям. Для этого используйте данные, полученные в предыдущем упражнении.

Ответ округлите до $2$-х знаков после запятой.